#### Testing combinations with starts and stops as indices

This is intended to be used with oamap version ( I know we don't intend to finally use it, but let's go with it for now).

##### Idea for calculation

Using the kernel structure used in test_combinations, we can  generate combinations arrays for every event. 

For that, each event is assigned a block ( i.e. each block in the grid serves as an event), and the combinations is calculated for each block using the `starts[block_index]` and event length.
This is appended to `left` and `right` arrays. 

In [1]:
import pycuda.autoinit
import pycuda
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
from pycuda.compiler import *
import numpy

In [2]:
# The generation step. Let's form a random integer array, from which we will form starts 
# and stops arrays
base_len = 32
base_arr = numpy.random.randint(6, size=base_len)

In [3]:
cumul_arr = numpy.cumsum(base_arr)

In [4]:
start = cumul_arr[:-1]
stop = cumul_arr[1:]

In [5]:
start

array([ 1,  4,  7, 11, 12, 15, 20, 25, 27, 28, 28, 29, 33, 34, 39, 40, 43,
       47, 52, 54, 58, 63, 66, 69, 74, 74, 77, 80, 81, 84, 85])

In [6]:
stop[:5]

array([ 4,  7, 11, 12, 15])

In [7]:
lengths = stop-start
pairs_lengths = numpy.zeros(len(lengths)+1, dtype=numpy.int32)
pairs_lengths[1:] = numpy.cumsum(lengths*lengths)
lengths_arr = numpy.array([base_len]).astype(numpy.int32)
lengths = lengths.astype(numpy.int32)

In [8]:
pairs_lengths

array([  0,   9,  18,  34,  35,  44,  69,  94,  98,  99,  99, 100, 116,
       117, 142, 143, 152, 168, 193, 197, 213, 238, 247, 256, 281, 281,
       290, 299, 300, 309, 310, 311], dtype=int32)

In [9]:
lengths

array([3, 3, 4, 1, 3, 5, 5, 2, 1, 0, 1, 4, 1, 5, 1, 3, 4, 5, 2, 4, 5, 3, 3,
       5, 0, 3, 3, 1, 3, 1, 1], dtype=int32)

##### Error 

The left and right arrays aren't changing at all, they remain as zeroes. Can't figure out why this is happening.

In [10]:
# Now let's form the cuda function

mod = SourceModule('''
__global__ void comb_events(int* left,int* right,int* start,int* length,int* lengths,int* pairs_lengths)
{
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    int j =  blockIdx.y*blockDim.y+threadIdx.y;
    int k =  blockIdx.z*blockDim.z+threadIdx.z;
    if (i <length[0] && j< lengths[i] && k<lengths[i])
    {
        left[pairs_lengths[i] + j*lengths[i] + k] = j + start[i];
        right[pairs_lengths[i] + j*lengths[i] + k] = k + start[i];
    }
}
''')

In [11]:
func = mod.get_function("comb_events")

In [12]:
left = numpy.zeros(pairs_lengths[-1]).astype(numpy.int32)
right = numpy.zeros(pairs_lengths[-1]).astype(numpy.int32)

In [13]:
func(cuda.InOut(left), cuda.InOut(right), cuda.In(start), cuda.In(lengths_arr),
    cuda.In(lengths), cuda.In(pairs_lengths), block=(1, 8, 8), grid=(base_len, 1))

In [16]:
left.reshape(pairs_lengths, base_len)
left

TypeError: only integer scalar arrays can be converted to a scalar index

In [15]:
right

array([ 1,  2,  3,  1,  2,  3,  1,  2,  3,  0,  1,  2,  0,  1,  2,  0,  1,
        2,  4,  5,  6,  7,  4,  5,  6,  7,  4,  5,  6,  7,  4,  5,  6,  7,
        0,  7,  8,  9,  7,  8,  9,  7,  8,  9,  0,  1,  2,  3,  4,  0,  1,
        2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,
        4, 11, 12, 13, 14, 15, 11, 12, 13, 14, 15, 11, 12, 13, 14, 15, 11,
       12, 13, 14, 15, 11, 12, 13, 14, 15,  0,  1,  0,  1, 12, 15,  0,  1,
        2,  3,  0,  1,  2,  3,  0,  1,  2,  3,  0,  1,  2,  3, 20,  0,  1,
        2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,
        4,  0,  1,  2,  3,  4, 25,  0,  1,  2,  0,  1,  2,  0,  1,  2, 27,
       28, 29, 30, 27, 28, 29, 30, 27, 28, 29, 30, 27, 28, 29, 30,  0,  1,
        2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,
        4,  0,  1,  2,  3,  4, 28, 29, 28, 29,  0,  1,  2,  3,  0,  1,  2,
        3,  0,  1,  2,  3,  0,  1,  2,  3, 28, 29, 30, 31, 32, 28, 29, 30,
       31, 32, 28, 29, 30